### QSE for the H2 molecule
Using Qiskit and Pyscf

In this notebook QSE algorithm (Quantum Subspace Expansion) is implemented and used to find both the first excited state and the ground state of the $H_{2}$ molecule.

The results of QSE are then compared to the classical solver of Pyscf, a lightweight, modular platform for quantum chemistry and solid-state calculations.

### Outline
<ol>
    <li>Required installs</li>
    <li>Algorithm steps</li>
    <li>Required imports</li>
    <li>Building the hamiltonian</li>
    <li>Building the ansatz and running VQE</li>
    <li>Building the excitation operators and subspace matrices</li>
    <li>Running the soluition</li>
    <li>Classical solution using Pyscf</li>
    <li>Conclusion</li>
    <li>References</li>
    <li>Versions</li>
</ol>

### Required installs
<hr>

The required installation steps are shown in the github Readme file also. If installed there then no need to reinstall here.

In [4]:
#!pip install qiskit['all']==1.4.4

In [5]:
#!pip install qiskit-nature-pyscf

In [6]:
#!pip install qiskit-aer

### Algorithm steps
<hr>

QSE is an algorithm for finding excited states of a given Hamiltonian $H$. It resembles the configuration interaction method in quantum chemistry.

1. ***Excitation Operators:***
Determine a set of excitation operators $E_1, \dots, E_M$ and a reference (approximate) ground state $|\psi_{GS}\rangle$. Single excitations of electrons, $c^\dagger_j c_l$ ($j, l = 0, 1, \dots$), are one of the common choices of the excitation operators. For notational convenience, we add $E_0 = I$ (identity operator) to the set of excitation operators. Note that $E_0 |\psi_{GS}\rangle = |\psi_{GS}\rangle$.

2. ***Ground state energy calculation:***
Prepare the approximate ground state $|\psi_{GS}\rangle$, obtained by the VQE algorithm (or other methods), on quantum computers.

3. ***Build the $S$ and $h$ matrices:***
Measure quantities $h_{ij} = \langle \psi_{GS} | E_i^\dagger H E_j | \psi_{GS} \rangle$ and $S_{ij} = \langle \psi_{GS} | E_i^\dagger E_j | \psi_{GS} \rangle$ on quantum computers ($i, j = 0, \dots, M$).

4. ***Diagonalization:***
Diagonalize the Hamiltonian within the subspace spanned by $E_0 |\psi_{GS}\rangle, \dots, E_M |\psi_{GS}\rangle$. Namely, solve a generalized eigenvalue problem within the subspace:  
$hC = SC E'$,  where $C$ is the coefficient vector for (approximate) eigenvectors and $E'$ is a diagonal matrix whose diagonal elements are (approximate) eigenvalues of $H$.

It is noted that the QSE also plays a role in mitigating noise errors inevitable in NISQ devices (see reference below).


### Required imports
<hr>

In [7]:
from qiskit_nature.second_q.drivers import PySCFDriver
from qiskit_nature.second_q.mappers import JordanWignerMapper
from qiskit_nature.second_q.circuit.library import UCCSD, HartreeFock
from qiskit_nature.second_q.operators import FermionicOp
from qiskit_algorithms import VQE
from qiskit_algorithms.optimizers import SLSQP
from qiskit.primitives import StatevectorEstimator
from qiskit.quantum_info import Statevector
from scipy.linalg import eigh
import numpy as np

from pyscf import gto, scf
from pyscf import fci

The following cell is to get rid of warnings like deprecated qiskit versions, for example.

In [8]:
import warnings
warnings.filterwarnings('ignore')

### Building the hamiltonian
<hr>

The $H_{2}$ hamiltonian is built by specifying the two constituting $H$ atoms with their coordinates in 3D.

In [9]:
# Step 1: Define H2 molecule and get Hamiltonian
driver = PySCFDriver(atom="H 0.0, 0.0, -0.6614; H 0.0, 0.0, 0.6614", basis="sto3g")
problem = driver.run()
mapper = JordanWignerMapper()
qubit_op = mapper.map(problem.hamiltonian.second_q_op())

In [10]:
qubit_op

SparsePauliOp(['IIII', 'IIIZ', 'IIZI', 'IIZZ', 'IZII', 'IZIZ', 'ZIII', 'ZIIZ', 'YYYY', 'XXYY', 'YYXX', 'XXXX', 'IZZI', 'ZIZI', 'ZZII'],
              coeffs=[-0.85624446+0.j,  0.10650235+0.j, -0.06094493+0.j,  0.09024315+0.j,
  0.10650235+0.j,  0.14377228+0.j, -0.06094493+0.j,  0.14472511+0.j,
  0.05448197+0.j,  0.05448197+0.j,  0.05448197+0.j,  0.05448197+0.j,
  0.14472511+0.j,  0.1514208 +0.j,  0.09024315+0.j])

### Building the ansatz and running VQE
<hr>

The eigenstate and eigenvalue got from the ground state will be used in the next steps for finding the excited states.

In [11]:
# Step 2: Build ansatz and run VQE
ansatz = UCCSD(
    problem.num_spatial_orbitals,
    problem.num_particles,
    mapper,
    initial_state=HartreeFock(
        problem.num_spatial_orbitals,
        problem.num_particles,
        mapper
    ),
)
vqe_solver = VQE(StatevectorEstimator(), ansatz, SLSQP())
vqe_result = vqe_solver.compute_minimum_eigenvalue(qubit_op)
params = vqe_result.optimal_parameters

### Building the excitation operators and subspace matrices
<hr>

Two important points are worth mentioning here.

1. Only single excited states operators will be built and used. Other operators like double excitation and spin operators for example can be built. But the simplest case of single excitations will be used for simplicity.

2. We have used the simulator to perform VQE in the previous cell, since we will not be using noisy simulators or running on real hardware, we can implement expectation values using tensors and multiplication in the following cell for finding the exited states. This is the same method used by the statevector simulator.

In [12]:
# Step 3: Generate single excitation operators
def generate_excitations(n_orb):
    ops = []
    for i in range(n_orb):
        for j in range(n_orb):
            if i != j:
                label = f"+_{i} -_{j}"
                ops.append(FermionicOp({label: 1.0}, num_spin_orbitals=n_orb))
    return ops

excitation_ops = generate_excitations(problem.num_spin_orbitals)
qubit_excitations = [mapper.map(op) for op in excitation_ops]

# Step 4: Prepare ground state vector
state = Statevector(ansatz.assign_parameters(params).decompose().to_instruction())

# Step 5: Build subspace matrices
def build_matrices(ops, H, psi):
    N = len(ops)
    S = np.zeros((N, N))
    H_mat = np.zeros((N, N))
    for i in range(N):
        psi_i = ops[i].to_matrix() @ psi.data
        for j in range(N):
            psi_j = ops[j].to_matrix() @ psi.data
            S[i, j] = np.vdot(psi_i, psi_j).real
            H_mat[i, j] = np.vdot(psi_i, H.to_matrix() @ psi_j).real
    return H_mat, S

H_sub, S_sub = build_matrices(qubit_excitations, qubit_op, state)

### Running the solution
<hr>

In [13]:
# Step 6: Solve generalized eigenvalue problem
epsilon = 1e-6 # epsilon value used as zero values are not allowed
S_reg = S_sub + epsilon * np.eye(S_sub.shape[0])
eigvals, eigvecs = eigh(H_sub, S_reg)
print("Excited state energies from QSE:", eigvals)

Excited state energies from QSE: [-1.26040021e+00 -1.26040021e+00 -1.26040006e+00 -8.24544918e-01
 -3.72699993e-01 -4.56105526e-11 -4.06214843e-11 -7.90068984e-17
 -5.13231016e-18  1.84893929e-17  6.71764436e-17  1.21031950e-11]


The first 3 values represent the ground state but with some degeneracy because only the single excitation operators were used. New operators enforce extra conditions so the number of values and degeneracy should decrease.

Nonetheless, the first 3 values represent the ground state energy -1.26 and the second value represents the first excited state energy -0.824

### Classical solution using pyscf:
<hr>

In [14]:
# Define H2 molecule
mol = gto.M(
    atom="H 0.0, 0.0, -0.6614; H 0.0, 0.0, 0.6614",
    basis="sto-3g",
    spin=0,
    charge=0
)

# Run Hartree-Fock
mf = scf.RHF(mol)
mf.kernel()

# Instantiate the FCI solver
fci_solver = fci.FCI(mf)

# Run Full CI (FCI)
fci_energies, _ = fci_solver.kernel(nroots=5)

print("🔬 FCI Energies (Hartree):")
for i, e in enumerate(fci_energies):
    print(f"  State {i}: {e:.6f}")

converged SCF energy = -0.965839207860032
🔬 FCI Energies (Hartree):
  State 0: -1.030503
  State 1: -0.860358
  State 2: -0.424502
  State 3: -0.231386


### Conclusion:
<hr>

In this notebook we have shown the following:
<ol>
    <li>How to build the hamiltonian operator of a molecule.</li>
    <li>How to calculate the ground state energy using VQE.</li>
    <li>How to calculate the first excited state using QSE.</li>
</ol>

\begin{array}{|c|c|c|}
\hline
\textbf{Algorithm} & \textbf{ground state energy} & \textbf{1st excited energy}\\
\hline
\text{QSE} & {-1.26} & -0.824\\
\text{Pyscf} & {-1.03} & -0.86\\
\hline
\end{array}

The ground state energy was a little near to the classical calculated solution but the value of the first excited state was very near and approximately equal to that of the classical solution.

### References
<hr>

<ol>
    <li>Pennylane VQE tutorial and H2 hamiltonian: <a href="https://pennylane.ai/qml/demos/tutorial_vqe">link</a></li>
    <li>Qiskit VQE: <a href="https://qiskit-community.github.io/qiskit-algorithms/tutorials/03_vqe_simulation_with_noise.html">link</a></li>
    <li>Algorithms for finding ground and excited states: <a href="https://quantaggle.com/algorithms/algorithm/">link</a></li>   
</ol>

### Versions
<hr>

In [15]:
import sys
print(sys.version)

3.12.11 (main, Jun  4 2025, 08:56:18) [GCC 11.4.0]


In [16]:
import pkg_resources

# Loop through installed packages and filter those starting with 'qiskit'
for dist in pkg_resources.working_set:
    if dist.project_name.lower().startswith("qiskit"):
        print(f"{dist.project_name} == {dist.version}")

qiskit == 1.4.4
qiskit-aer == 0.17.1
qiskit-algorithms == 0.4.0
qiskit-nature == 0.7.2
qiskit-nature-pyscf == 0.4.0
qiskit-qasm3-import == 0.6.0
